# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [ ]:
## Before the project, analyzed by Leon Wang
## Please double check the required output.

# How to decide the algorithm?
# Based on the info, 
# """In the Project Workspace, you'll find a data set containing real messages that were sent during disaster events. 
# You will be creating a machine learning pipeline to categorize these events 
#   so that you can send the messages to an appropriate disaster relief agency."""
# this project is designed to analyze/learn messages while in disaster happened
# @ text/message problem
# possible algorithm and/or methods
# 1. NLTK



In [ ]:
# Project Workspace - ETL
# The first part of your data pipeline is the Extract, Transform, and Load process. 
# Here, you will read the dataset, clean the data, and then store it in a SQLite database. 
# We expect you to do the data cleaning with pandas. 
# To load the data into an SQLite database, you can use the pandas dataframe .to_sql() method, 
#  which you can use with an SQLAlchemy engine.

# Feel free to do some exploratory data analysis in order to figure out how you want to clean the data set. 
# Though you do not need to submit this exploratory data analysis as part of your project, 
# you'll need to include your cleaning code in the final ETL script, process_data.py.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

In [2]:
# check existence of null values in columns
def check_nan(obj, col): 
    if (obj[col].isnull().sum() > 0):
        print("total # of lines: ", obj.shape[0])
    print("count # of isnan in ", col, ": ", obj[col].isnull().sum(), 
          "\ncount # of isval in ", col, ": ", obj[col].notnull().sum(),"\n")



In [3]:
# load messages dataset
file_msg = "messages.csv"
messages = pd.read_csv(file_msg)

cols = messages.columns.values
print("# of data lines:", messages.shape[0],"\n# of data columns: ", messages.shape[1])
print("Columns of \"", file_msg, "\" are ", cols)

print("\n_____________________\n")
for col in cols:
    check_nan(messages, col)

print("_____________________\n")

messages.head()


# of data lines: 26248 
# of data columns:  4
Columns of " messages.csv " are  ['id' 'message' 'original' 'genre']

_____________________

count # of isnan in  id :  0 
count # of isval in  id :  26248 

count # of isnan in  message :  0 
count # of isval in  message :  26248 

total # of lines:  26248
count # of isnan in  original :  16064 
count # of isval in  original :  10184 

count # of isnan in  genre :  0 
count # of isval in  genre :  26248 

_____________________



,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
# load categories dataset
file_ctg = "categories.csv"
categories = pd.read_csv(file_ctg)
cols = categories.columns.values

print("# of data lines:", categories.shape[0],"\n# of data columns: ", categories.shape[1])
print("Columns of \"", file_ctg, "\" are ", cols)
print("_____________________\n")
for col in cols:
    check_nan(categories, col)
print("_____________________\n")
categories.head()

# of data lines: 26248 
# of data columns:  2
Columns of " categories.csv " are  ['id' 'categories']
_____________________

count # of isnan in  id :  0 
count # of isval in  id :  26248 

count # of isnan in  categories :  0 
count # of isval in  categories :  26248 

_____________________



,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
# merge datasets
df = messages.merge(categories, how="left", on=["id"])
df.head()
# df.tail()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [6]:
# create a dataframe of the 36 individual category columns

# categories_tmp = categories["categories"].str.split(";")
# categories["categories"] = categories["categories"].str.split(pat=";", expand=True)
# df["categories"].str.split(";")
# categories["categories"].str.split(pat=";")
categories = categories["categories"].str.split(pat=";", expand=True)
# categories = categories.str.split(pat=";",expand=True)
categories.head(5)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [7]:
# select the first row of the categories dataframe
#method #1
row = categories.iloc[0]
# print(row)
# print(categories.iloc[1])
# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [category_name.split("-")[0] for category_name in row.values]

print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [ ]:
# # select the first row of the categories dataframe
# #method #2
# row = categories[0:1]
# # print(row[0])
# # print(categories.iloc[1])
# # use this row to extract a list of new column names for categories.
# # one way is to apply a lambda function that takes everything 
# # up to the second to last character of each string with slicing 
# category_colnames = row.apply(lambda x: x.str[:-2]).values.tolist()
# # category_colnames = row.apply(lambda x: x.str.split("-")[0] for category_name in row.values).values.tolist()
# print(category_colnames)

In [8]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [9]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
#     categories[column] = pd.to_numeric(categories[column])
    categories[column] = categories[column].astype(int)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [10]:
# drop the original categories column from `df`
df.drop(['categories'], axis=1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [11]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df,categories],join="inner", axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [12]:
# check number of duplicates
print("Number of data with duplicates is {}.".format(len(df.index)))
print("Nmber of duplicated data is {}.".format(sum(df.duplicated())))
print(df[df.duplicated()])

Number of data with duplicates is 26248.
Nmber of duplicated data is 41.
          id                                            message  \
716      862  What is the address of the radio station? I as...   
1414    1652  please we need water, food and tents, we have ...   
1416    1652  please we need water, food and tents, we have ...   
4653    5265  (Delmas 33 Charboniere infomatyon s'il vous pl...   
4978    5643  Good evening. we are writing to ask you for a ...   
4979    5643  Good evening. we are writing to ask you for a ...   
5102    5776  I would like some more informations about trav...   
5588    6327  jel2  Acte 5: 29 2chr7 Lev11 Ecl9 Ecl4 jos5 ch...   
5589    6327  jel2  Acte 5: 29 2chr7 Lev11 Ecl9 Ecl4 jos5 ch...   
5639    6393    I'd like to sign up in the 4636 program please    
5641    6393    I'd like to sign up in the 4636 program please    
5731    6492           I'd like to know an email to find a job    
5756    6515  I would like to find some informations abo

In [13]:
# drop duplicates
print("Number of data with duplicates is {}.".format(len(df.index)))
print("drop duplicates")
df.drop_duplicates(inplace=True)
print("Number of data after removal is {}.".format(len(df.index)))

Number of data with duplicates is 26248.
drop duplicates
Number of data after removal is 26207.


In [14]:
# check number of duplicates
print("Number of duplicates after removal is {}.".format(sum(df.duplicated())))

Number of duplicates after removal is 0.


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [15]:
database_path = "disaster_response_db.db"
engine = create_engine('sqlite:///'+ database_path)
table_name = os.path.basename(database_path).replace('.db','')+'_table'
print(table_name)
df.to_sql(table_name, engine, index=False, if_exists='replace')

disaster_response_db_table


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.